28 Febuary 2024
# <center>Lab 7 Assignment - CS 4315<center>
<center>Doug Andrade</center>

#### 1. Load 'armada.csv' and convert all categorical columns to either binary or one-hot vectors, as requested previously and excluding 'Battle', and z-score all columns

In [1]:
# Import the Pandas module for data frame operations
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

# Set the csv file name to a object, to avoid hard-coding
csv_file = 'armada.csv'

# Read-in the csv file as a Pandas data frame, as an object to be operated on later
armada_df = pd.read_csv(filepath_or_buffer = csv_file)

armada_df.head(3)

,Battle,Year,Portuguese Ships,Dutch Ships,English Ships,Ratio of Portuguese Ships to Dutch/British Ships,Spanish Involvement,Portuguese Outcome
0,Bantam,1601,6,3,0,2.000,No,Draw
1,Malacca Strait,1606,14,11,0,1.273,No,Draw
2,Ilha das Naus,1606,6,9,0,0.667,No,Defeat


In [2]:
# Drop the "Battle" column
armada_train_df = armada_df.drop(columns = ['Battle'], axis = 1)

# List all non-numeric columns
categorical_col_list = [col for col in armada_train_df.columns if armada_train_df[col].dtype == 'object']

In [3]:
# Loop through categorical columns
for col in categorical_col_list:
    # Check of binary columns (two categories)
    if armada_train_df[col].nunique() < 3:
        # Create the new binary column name
        binary_col_name = col + ' Binary'
        # Build out the new binary column, without replacing the orginal
        armada_train_df[binary_col_name] = armada_train_df[col].replace({'No': 0, 'Yes': 1})
    # Continue looping through all non-numeric columns with 3 or more categories
    else:
        # concatenate the one-hot encoded columns to the original dataframe, setting drop_first to False to prevent
        # unintended correlations
        armada_train_df = pd.concat(objs = [armada_train_df, 
                                            pd.get_dummies(armada_train_df[col],
                                                           prefix = col,
                                                           drop_first = False)], 
                                    axis = 1)

armada_train_df.head(3)

,Year,Portuguese Ships,Dutch Ships,English Ships,Ratio of Portuguese Ships to Dutch/British Ships,Spanish Involvement,Portuguese Outcome,Spanish Involvement Binary,Portuguese Outcome_Defeat,Portuguese Outcome_Draw,Portuguese Outcome_Victory
0,1601,6,3,0,2.000,No,Draw,0,False,True,False
1,1606,14,11,0,1.273,No,Draw,0,False,True,False
2,1606,6,9,0,0.667,No,Defeat,0,True,False,False


In [4]:
# Using list comprehension and conditional statements, create a list of all the numeric column names
numeric_col_list = [col for col in armada_train_df.columns if armada_train_df[col].dtype != 'object']

In [5]:
armada_zscore = armada_train_df

# Iterate only the data frame columns that exist in the "numeric_col_list" above
for col in numeric_col_list:
    # Create a new column name for the cooresponding columns Z-Score
    zscored_col_name = col + ' Z-Score'
    # Immediately insert the new Z-Score column next to the parent column by adding one to the parent's column position 
    # within the insert function
    armada_zscore.insert(armada_zscore.columns.get_loc(col) + 1, 
                     # Calculate the Z-Score (x - x_bar) / x_std)
                           zscored_col_name, 
                           (armada_zscore[col] - armada_zscore[col].mean()) / armada_zscore[col].std())

armada_zscore.head(3)

,Year,Year Z-Score,Portuguese Ships,Portuguese Ships Z-Score,Dutch Ships,Dutch Ships Z-Score,English Ships,English Ships Z-Score,Ratio of Portuguese Ships to Dutch/British Ships,Ratio of Portuguese Ships to Dutch/British Ships Z-Score,Spanish Involvement,Portuguese Outcome,Spanish Involvement Binary,Spanish Involvement Binary Z-Score,Portuguese Outcome_Defeat,Portuguese Outcome_Defeat Z-Score,Portuguese Outcome_Draw,Portuguese Outcome_Draw Z-Score,Portuguese Outcome_Victory,Portuguese Outcome_Victory Z-Score
0,1601,-1.560039,6,-0.448594,3,-0.468067,0,-0.301249,2.000,0.904955,No,Draw,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
1,1606,-1.275286,14,0.053831,11,-0.109002,0,-0.301249,1.273,0.121838,No,Draw,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
2,1606,-1.275286,6,-0.448594,9,-0.198768,0,-0.301249,0.667,-0.530939,No,Defeat,0,-0.914174,True,1.317465,False,-0.914174,False,-0.457851


#### 2. Use those columns to predict the binary 'Spanish Involvement' for the ex1 example using k-nearest neighbors, z-scoring the example vector using the statistics from 'armada.csv'.

In [6]:
armada_ex1 = pd.read_csv(filepath_or_buffer = 'armada_ex1.csv')
armada_ex1

,Year,Portuguese Ships,Dutch Ships,English Ships,Ratio of Portuguese Ships to Dutch/British Ships,Portuguese Outcome
0,1605,8,1,3,2,Draw


In [7]:
# Create boolean columns based on 'Portuguese Outcome'
armada_ex1['Portuguese Outcome_Defeat'] = False
armada_ex1['Portuguese Outcome_Draw'] = False
armada_ex1['Portuguese Outcome_Victory'] = False

# Use loc to set the correct boolean value based on the condition
armada_ex1.loc[armada_ex1['Portuguese Outcome'] == 'Defeat', 'Portuguese Outcome_Defeat'] = True
armada_ex1.loc[armada_ex1['Portuguese Outcome'] == 'Draw', 'Portuguese Outcome_Draw'] = True
armada_ex1.loc[armada_ex1['Portuguese Outcome'] == 'Victory', 'Portuguese Outcome_Victory'] = True

armada_ex1

,Year,Portuguese Ships,Dutch Ships,English Ships,Ratio of Portuguese Ships to Dutch/British Ships,Portuguese Outcome,Portuguese Outcome_Defeat,Portuguese Outcome_Draw,Portuguese Outcome_Victory
0,1605,8,1,3,2,Draw,False,True,False


In [8]:
armada_zscore_numeric_col = [col for col in armada_zscore.columns if armada_zscore[col].dtype != 'object']

armada_df_mean = armada_zscore[armada_zscore_numeric_col].mean()
armada_df_std = armada_zscore[armada_zscore_numeric_col].std()

In [9]:
armada_ex1_zscore = armada_ex1
armada_ex1_numeric_col = [col for col in armada_ex1.columns if armada_ex1[col].dtype != 'object']

# Iterate only the data frame columns that exist in the "numeric_col_list" above
for col in armada_ex1_numeric_col:
    # Create a new column name for the cooresponding columns Z-Score
    zscored_col_name = col + ' Z-Score'
    # Immediately insert the new Z-Score column next to the parent column by adding one to the parent's column position 
    # within the insert function
    armada_ex1_zscore.insert(armada_ex1_zscore.columns.get_loc(col) + 1,
                             # Calculate the Z-Score (x - x_bar) / x_std)
                             zscored_col_name,
                             (armada_ex1_zscore[col] - armada_df_mean[col]) / armada_df_std[col])
armada_ex1_zscore

,Year,Year Z-Score,Portuguese Ships,Portuguese Ships Z-Score,Dutch Ships,Dutch Ships Z-Score,English Ships,English Ships Z-Score,Ratio of Portuguese Ships to Dutch/British Ships,Ratio of Portuguese Ships to Dutch/British Ships Z-Score,Portuguese Outcome,Portuguese Outcome_Defeat,Portuguese Outcome_Defeat Z-Score,Portuguese Outcome_Draw,Portuguese Outcome_Draw Z-Score,Portuguese Outcome_Victory,Portuguese Outcome_Victory Z-Score
0,1605,-1.332237,8,-0.322988,1,-0.557833,3,0.20485,2,0.904955,Draw,False,-0.731925,True,1.054816,False,-0.457851


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

X_train = armada_zscore[[col for col in armada_zscore.columns if armada_zscore[col].dtype != 'object' and armada_zscore[col].dtype != 'bool']]
X_train = X_train.drop(columns=['Spanish Involvement Binary', 'Spanish Involvement Binary Z-Score'], axis=1)

y_train = armada_zscore['Spanish Involvement Binary']

knn_model = KNeighborsClassifier(n_neighbors = 3)  # You can adjust the number of neighbors
knn_model.fit(X_train, y_train)

x_pred = armada_ex1_zscore[[col for col in armada_ex1_zscore.columns if armada_ex1_zscore[col].dtype != 'object' and armada_ex1_zscore[col].dtype != 'bool']]
armada_ex1_prediction = knn_model.predict(x_pred)

print('Using KNN classification, the prediction of Spanish involvement is: %s' % ('No' if armada_ex1_prediction == 0 else 'Yes'))

Using KNN classification, the prediction of Spanish involvement is: No


#### 3. Use those columns to predict 'Portuguese Ships' for the ex2 example using k-nearest neighbors, z-scoring the example vector using the statistics from 'armada.csv'.

In [11]:
armada_ex2 = pd.read_csv(filepath_or_buffer = 'armada_ex2.csv')
armada_ex2

,Year,Dutch Ships,English Ships,Spanish Involvement,Portuguese Outcome
0,1645,9,3,Yes,Victory


In [12]:
# List all non-numeric columns
armada_ex2_cat_col_list = ['Spanish Involvement']

# Loop through categorical columns
for col in armada_ex2_cat_col_list:
    # Check of binary columns (two categories)
    if armada_ex2[col].nunique() < 3:
        # Create the new binary column name
        binary_col_name = col + ' Binary'
        # Build out the new binary column, without replacing the orginal
        armada_ex2[binary_col_name] = armada_ex2[col].replace({'No': 0, 'Yes': 1})
    # Continue looping through all non-numeric columns with 3 or more categories
    else:
        # concatenate the one-hot encoded columns to the original dataframe, setting drop_first to False to prevent
        # unintended correlations
        armada_ex2 = pd.concat(objs = [armada_ex2, 
                                            pd.get_dummies(armada_ex2[col],
                                                           prefix = col,
                                                           drop_first = False)], 
                                    axis = 1)
armada_ex2

,Year,Dutch Ships,English Ships,Spanish Involvement,Portuguese Outcome,Spanish Involvement Binary
0,1645,9,3,Yes,Victory,1


In [13]:
# Create boolean columns based on 'Portuguese Outcome'
armada_ex2['Portuguese Outcome_Defeat'] = False
armada_ex2['Portuguese Outcome_Draw'] = False
armada_ex2['Portuguese Outcome_Victory'] = False

# Use loc to set the correct boolean value based on the condition
armada_ex2.loc[armada_ex2['Portuguese Outcome'] == 'Defeat', 'Portuguese Outcome_Defeat'] = True
armada_ex2.loc[armada_ex2['Portuguese Outcome'] == 'Draw', 'Portuguese Outcome_Draw'] = True
armada_ex2.loc[armada_ex2['Portuguese Outcome'] == 'Victory', 'Portuguese Outcome_Victory'] = True

armada_ex2

,Year,Dutch Ships,English Ships,Spanish Involvement,Portuguese Outcome,Spanish Involvement Binary,Portuguese Outcome_Defeat,Portuguese Outcome_Draw,Portuguese Outcome_Victory
0,1645,9,3,Yes,Victory,1,False,False,True


In [14]:

armada_ex2_zscore = armada_ex2
armada_ex2_numeric_col = [col for col in armada_ex2.columns if armada_ex2[col].dtype != 'object']

# Iterate only the data frame columns that exist in the "numeric_col_list" above
for col in armada_ex2_numeric_col:
    # Create a new column name for the cooresponding columns Z-Score
    zscored_col_name = col + ' Z-Score'
    # Immediately insert the new Z-Score column next to the parent column by adding one to the parent's column position 
    # within the insert function
    armada_ex2_zscore.insert(armada_ex2_zscore.columns.get_loc(col) + 1,
                             # Calculate the Z-Score (x - x_bar) / x_std)
                             zscored_col_name,
                             (armada_ex2_zscore[col] - armada_df_mean[col]) / armada_df_std[col])
armada_ex2_zscore

,Year,Year Z-Score,Dutch Ships,Dutch Ships Z-Score,English Ships,English Ships Z-Score,Spanish Involvement,Portuguese Outcome,Spanish Involvement Binary,Spanish Involvement Binary Z-Score,Portuguese Outcome_Defeat,Portuguese Outcome_Defeat Z-Score,Portuguese Outcome_Draw,Portuguese Outcome_Draw Z-Score,Portuguese Outcome_Victory,Portuguese Outcome_Victory Z-Score
0,1645,0.945786,9,-0.198768,3,0.20485,Yes,Victory,1,1.054816,False,-0.731925,False,-0.914174,True,2.106114


In [15]:
X_train_3 = armada_zscore[[col for col in armada_zscore.columns if armada_zscore[col].dtype != 'object' and armada_zscore[col].dtype != 'bool']]
X_train_3 = X_train_3.drop(columns=['Portuguese Ships',
                                  'Portuguese Ships Z-Score',
                                  'Ratio of Portuguese Ships to Dutch/British Ships',
                                  'Ratio of Portuguese Ships to Dutch/British Ships Z-Score'], axis=1)

y_train_3 = armada_zscore['Portuguese Ships']

knn_regressor_euclidean = KNeighborsRegressor(n_neighbors = 3, weights = 'uniform', metric = 'euclidean')
knn_regressor_euclidean.fit(X_train_3, y_train_3)

x_pred_3 = armada_ex2_zscore[[col for col in armada_ex2_zscore.columns if armada_ex2_zscore[col].dtype != 'object' and armada_ex2_zscore[col].dtype != 'bool']]
armada_ex2_prediction = knn_regressor_euclidean.predict(x_pred_3)

print('Using KNN regression, the prediction of Portugese ships is: %s' % armada_ex2_prediction)

Using KNN regression, the prediction of Portugese ships is: [21.]


#### 4. Use ROSE and SMOTE to oversample the z-score, binary, and one-hot columns based on 'Spanish Involvement'

In [16]:
import imblearn

In [17]:
armada_zscore['Spanish Involvement'].value_counts()

Spanish Involvement
No     15
Yes    13
Name: count, dtype: int64

In [18]:
features = ['Year Z-Score', 
            'Portuguese Ships Z-Score',
            'Dutch Ships Z-Score',
            'English Ships Z-Score',
            'Ratio of Portuguese Ships to Dutch/British Ships Z-Score',
            'Spanish Involvement Binary',
            'Spanish Involvement Binary Z-Score',
            'Portuguese Outcome_Defeat',
            'Portuguese Outcome_Defeat Z-Score',
            'Portuguese Outcome_Draw',
            'Portuguese Outcome_Draw Z-Score',
            'Portuguese Outcome_Victory',
            'Portuguese Outcome_Victory Z-Score']

In [19]:
rose = imblearn.over_sampling.RandomOverSampler(sampling_strategy = 'auto', 
                                                random_state = None, 
                                                shrinkage = 1.)

In [20]:
# ROSE
df_x_rose, df_y_rose = rose.fit_resample(armada_zscore[features],
                                         armada_zscore['Spanish Involvement'])

In [21]:
# Check the for Spanish Involvement balance
df_y_rose.value_counts()

Spanish Involvement
No     15
Yes    15
Name: count, dtype: int64

In [22]:
# Review the ROSE output
pd.concat([df_y_rose, df_x_rose], axis = 1).head(3)

,Spanish Involvement,Year Z-Score,Portuguese Ships Z-Score,Dutch Ships Z-Score,English Ships Z-Score,Ratio of Portuguese Ships to Dutch/British Ships Z-Score,Spanish Involvement Binary,Spanish Involvement Binary Z-Score,Portuguese Outcome_Defeat,Portuguese Outcome_Defeat Z-Score,Portuguese Outcome_Draw,Portuguese Outcome_Draw Z-Score,Portuguese Outcome_Victory,Portuguese Outcome_Victory Z-Score
0,No,-1.560039,-0.448594,-0.468067,-0.301249,0.904955,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
1,No,-1.275286,0.053831,-0.109002,-0.301249,0.121838,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
2,No,-1.275286,-0.448594,-0.198768,-0.301249,-0.530939,0,-0.914174,True,1.317465,False,-0.914174,False,-0.457851


In [23]:
# SMOTE
df_x_smote, df_y_smote = imblearn.over_sampling.SMOTE().fit_resample(armada_zscore[features],
                                                                     armada_zscore['Spanish Involvement'])

In [24]:
# Check the for Spanish Involvement balance
df_y_smote.value_counts()

Spanish Involvement
No     15
Yes    15
Name: count, dtype: int64

In [25]:
# Review the SMOTE output
pd.concat([df_y_smote, df_x_smote], axis = 1).head(3)

,Spanish Involvement,Year Z-Score,Portuguese Ships Z-Score,Dutch Ships Z-Score,English Ships Z-Score,Ratio of Portuguese Ships to Dutch/British Ships Z-Score,Spanish Involvement Binary,Spanish Involvement Binary Z-Score,Portuguese Outcome_Defeat,Portuguese Outcome_Defeat Z-Score,Portuguese Outcome_Draw,Portuguese Outcome_Draw Z-Score,Portuguese Outcome_Victory,Portuguese Outcome_Victory Z-Score
0,No,-1.560039,-0.448594,-0.468067,-0.301249,0.904955,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
1,No,-1.275286,0.053831,-0.109002,-0.301249,0.121838,0,-0.914174,False,-0.731925,True,1.054816,False,-0.457851
2,No,-1.275286,-0.448594,-0.198768,-0.301249,-0.530939,0,-0.914174,True,1.317465,False,-0.914174,False,-0.457851
